<a href="https://colab.research.google.com/github/EWHA-AI24-Project/Weather-to-Wildfire-Prediction/blob/main/notebooks/model_2/model2_tuning_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, roc_auc_score, average_precision_score
import numpy as np
from scipy.stats import randint


# 데이터 로드
data = pd.read_csv('Data2_train.csv')
X = data.drop(columns=['FIRE_START_DAY'])
y = data['FIRE_START_DAY']

# 트레인/검증 분할
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# y_train, y_val을 명시적으로 int형으로 변환 (0, 1)
y_train = y_train.astype(int)
y_val = y_val.astype(int)

In [ ]:
# 커스텀 점수 함수
def custom_score(y_true, y_pred_proba, **kwargs):
    pr_auc = average_precision_score(y_true, y_pred_proba)
    roc_auc = roc_auc_score(y_true, y_pred_proba)
    return 0.5 * pr_auc + 0.5 * roc_auc

# Scorer 객체로 변환
custom_scorer = make_scorer(custom_score, needs_proba=True)

In [ ]:
# === 첫 번째 랜덤서치 ===

# 랜덤포레스트 모델 정의
rf = RandomForestClassifier(random_state=42)

param_dist = {
    'n_estimators': [120, 150, 180, 200, 220],
    'max_depth': [8, 10, 15, 20],
    'min_samples_split': [8, 10, 12, 15],
    'min_samples_leaf': [3, 4, 5],
    'max_features': ['sqrt', 'log2']
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Random Search
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=90,
    scoring=custom_scorer,
    cv=cv,
    verbose=1,
    random_state=42,
    n_jobs=-1,
    error_score='raise'
)

random_search.fit(X_train, y_train)


# 최적 모델로 검증셋 평가

best_model = random_search.best_estimator_
y_val_proba = best_model.predict_proba(X_val)[:, 1]

pr_auc = average_precision_score(y_val, y_val_proba)
roc_auc = roc_auc_score(y_val, y_val_proba)
custom = 0.5 * pr_auc + 0.5 * roc_auc

# 출력
print("✅ Best CV Score (Custom):", random_search.best_score_)
print("📌 Validation PR AUC :", pr_auc)
print("📌 Validation ROC AUC:", roc_auc)
print("🎯 Custom Score (0.5 * PR AUC + 0.5 * ROC AUC):", custom)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
✅ Best CV Score (Custom): 0.6360986041281598
📌 Validation PR AUC : 0.6723653258141535
📌 Validation ROC AUC: 0.8285485364374333
🎯 Custom Score (0.5 * PR AUC + 0.5 * ROC AUC): 0.7504569311257934


In [ ]:
print("Best parameters:", random_search.best_params_)

Best parameters: {'n_estimators': 120, 'min_samples_split': 15, 'min_samples_leaf': 5, 'max_features': 'sqrt', 'max_depth': 10}


In [ ]:
# === 두 번째 랜덤서치 ===

# 랜덤포레스트 모델 정의
rf = RandomForestClassifier(random_state=42)

param_dist = {
    'n_estimators': [100, 120, 140, 160, 180],
    'max_depth': [8, 10, 12, 14],
    'min_samples_split': [12, 15, 18, 21],
    'min_samples_leaf': [4, 5, 6],
    'max_features': ['sqrt', 'log2']
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 4. 랜덤서치 실행
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=90,
    scoring=custom_scorer,
    cv=cv,
    verbose=1,
    random_state=42,
    n_jobs=-1,
    error_score='raise'
)

random_search.fit(X_train, y_train)


# 최적 모델로 검증셋 평가

best_model = random_search.best_estimator_
y_val_proba = best_model.predict_proba(X_val)[:, 1]

pr_auc = average_precision_score(y_val, y_val_proba)
roc_auc = roc_auc_score(y_val, y_val_proba)
custom = 0.5 * pr_auc + 0.5 * roc_auc

# 출력 (파라미터 + 성능)
print("Best parameters:", random_search.best_params_)

print("✅ Best CV Score (Custom):", random_search.best_score_)
print("📌 Validation PR AUC :", pr_auc)
print("📌 Validation ROC AUC:", roc_auc)
print("🎯 Custom Score (0.5 * PR AUC + 0.5 * ROC AUC):", custom)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
Best parameters: {'n_estimators': 180, 'min_samples_split': 18, 'min_samples_leaf': 5, 'max_features': 'log2', 'max_depth': 12}
✅ Best CV Score (Custom): 0.6378315187411078
📌 Validation PR AUC : 0.6704822601257405
📌 Validation ROC AUC: 0.8264583508695471
🎯 Custom Score (0.5 * PR AUC + 0.5 * ROC AUC): 0.7484703054976438


In [ ]:
# === 세 번째 랜덤서치 ===

# 랜덤포레스트 모델 정의
rf = RandomForestClassifier(random_state=42)

param_dist = {
    'n_estimators': [120, 150, 180, 210],
    'max_depth': [10,12,14],
    'min_samples_split': [15, 18, 21],
    'min_samples_leaf': [4, 5, 6],
    'max_features': ['sqrt', 'log2']
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 4. 랜덤서치 실행
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=90,
    scoring=custom_scorer,
    cv=cv,
    verbose=1,
    random_state=42,
    n_jobs=-1,
    error_score='raise'
)

random_search.fit(X_train, y_train)


# 최적 모델로 검증셋 평가

best_model = random_search.best_estimator_
y_val_proba = best_model.predict_proba(X_val)[:, 1]

pr_auc = average_precision_score(y_val, y_val_proba)
roc_auc = roc_auc_score(y_val, y_val_proba)
custom = 0.5 * pr_auc + 0.5 * roc_auc

#  출력 (파라미터 + 성능)
print("Best parameters:", random_search.best_params_)

print("✅ Best CV Score (Custom):", random_search.best_score_)
print("📌 Validation PR AUC :", pr_auc)
print("📌 Validation ROC AUC:", roc_auc)
print("🎯 Custom Score (0.5 * PR AUC + 0.5 * ROC AUC):", custom)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
Best parameters: {'n_estimators': 150, 'min_samples_split': 18, 'min_samples_leaf': 5, 'max_features': 'log2', 'max_depth': 12}
✅ Best CV Score (Custom): 0.6362435102823832
📌 Validation PR AUC : 0.6707795577910767
📌 Validation ROC AUC: 0.8266353519055931
🎯 Custom Score (0.5 * PR AUC + 0.5 * ROC AUC): 0.7487074548483349


In [ ]:
# === 그리드 서치 ===

# 모델 정의
rf = RandomForestClassifier(random_state=42)

# Parameter range refinement based on random search results
param_grid = {
    'n_estimators': [120, 150, 180, 200, 230],
    'max_depth': [10, 12, 15, 18],
    'min_samples_split': [12, 15, 18, 20],
    'min_samples_leaf': [4, 5, 6],
    'max_features': ['sqrt', 'log2']
}

# 같은 교차검증 방식 사용
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring=custom_scorer,
    cv=cv,
    n_jobs=-1,
    verbose=1,
    error_score='raise'
)

grid_search.fit(X_train, y_train)

# === 최적 모델로 검증셋 평가 ===

best_model = grid_search.best_estimator_
y_val_proba = best_model.predict_proba(X_val)[:, 1]

# 지표 계산
pr_auc = average_precision_score(y_val, y_val_proba)
roc_auc = roc_auc_score(y_val, y_val_proba)
custom = 0.5 * pr_auc + 0.5 * roc_auc

# 출력
print("Best Parameters:", grid_search.best_params_)
print("✅ Best CV Score (Custom):", grid_search.best_score_)
print("📌Validation PR AUC :", pr_auc)
print("📌Validation ROC AUC:", roc_auc)
print("🎯Custom Score (0.5 * PR AUC + 0.5 * ROC AUC):", custom)

Fitting 5 folds for each of 480 candidates, totalling 2400 fits
Best Parameters: {'max_depth': 12, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'min_samples_split': 18, 'n_estimators': 180}
✅ Best CV Score (Custom): 0.6378315187411078
📌Validation PR AUC : 0.6704822601257405
📌Validation ROC AUC: 0.8264583508695471
🎯Custom Score (0.5 * PR AUC + 0.5 * ROC AUC): 0.7484703054976438


In [ ]:
# 첫 번째 랜덤서치 모델로 선택